In [257]:
import pandas as pd
import plotly.express as px
import plotly
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Data on EV% of new sales
Copied from https://www.eafo.eu/vehicles-and-fleet/m1#

In [35]:
years = list(range(2010, 2021))
print(years)

[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]


In [36]:
percent_share = [.3, 1.5, 3.1, 5.8, 12.6, 17.1, 15.7, 20.8, 31.2, 42.3, 51.6]

In [37]:
df = pd.DataFrame(index=years, data=percent_share, columns=['% Newly Registered Cars'])

In [38]:
fig = px.area(df, x=df.index, y='% Newly Registered Cars', template='plotly_white',
)
fig.update_traces(line=dict(width=3), mode='lines+markers')
fig.update_xaxes(showgrid=False)

fig.add_shape(x0=2010, x1=2020, y0=50, y1=50, line=dict(color='red', width=4),opacity=.2)

# Data on average CO2 intensity of new cars
https://en.wikipedia.org/wiki/Plug-in_electric_vehicles_in_Norway#/media/File:Average_fleet_CO2_emissions_Norway.png

In [39]:
years = range(2011, 2020)
data = [131,130, 123, 110, 100, 93,82, 71, 60]
df_fleet = pd.DataFrame(index=years, data=data, columns=['Avg fleet intensity'])

In [40]:
df = df.join(df_fleet, how='left')

We don't have data for 2020, fit a regression

In [43]:
from sklearn.linear_model import LinearRegression

In [44]:
model = LinearRegression()

In [51]:
np.reshape(df.index.values, (-1,))

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [55]:
model.fit(np.reshape(df.dropna().index.values, (-1,1)),
          np.reshape(df.dropna()['Avg fleet intensity'].values, (-1, 1)))

LinearRegression()

In [60]:
value_2020 = model.predict([[2020]])

In [61]:
df.loc[2020, 'Avg fleet intensity'] = value_2020[0][0]

In [297]:
df = df.loc[2011:]

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(x=df.index.values, y=df['Avg fleet intensity'], name="New Fleet Intensity, gCO2/km", opacity=.5,
           width=.2,
          marker=dict(color=df['Avg fleet intensity'], colorscale='darkmint', line=dict(width=0),
                     ),
          )
    ,
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df.index.values, y=df['% Newly Registered Cars'], name="yaxis data", 
                  marker=dict(color='blue')),
    secondary_y=True,


)


fig.add_shape(x0=2010,x1=2020, y0=59.9, y1=59.9, line=dict(color='green'))

target_2030 = 59.4
target_2025 = 80.8
target_2021 = 95

fig.add_shape(x0=2010,x1=2020, y0=target_2021, y1=target_2021, line=dict(color='green'), name='EU Target 2021',
                         opacity=.3)

fig.add_annotation(text="EU Target 2021", x=2009, y=target_2021, showarrow=False)
                   
fig.add_shape(x0=2010,x1=2020, y0=target_2025, y1=target_2025, line=dict(color='green'), name='EU Target 2025',
                         opacity=.5)

fig.add_annotation(text="EU Target 2025", x=2009, y=target_2025, showarrow=False)

fig.add_shape(x0=2010,x1=2020, y0=target_2030, y1=target_2030, line=dict(color='green'), name='EU Target 2030',
             opacity=.7)
fig.add_annotation(text="EU Target 2030", x=2009, y=target_2030, showarrow=False)


fig.update_layout(template='plotly_white', showlegend=False)


fig.update_yaxes(showgrid=False, secondary_y=True)
fig.update_yaxes(showgrid=False, showticklabels=False)




In [178]:
['black' if x > 50 else 'gray' for x in df['% Newly Registered Cars'].values]

['gray',
 'gray',
 'gray',
 'gray',
 'gray',
 'gray',
 'gray',
 'gray',
 'gray',
 'gray',
 'black']

In [300]:
df

,% Newly Registered Cars,Avg fleet intensity
2011,1.5,131.000000
2012,3.1,130.000000
2013,5.8,123.000000
2014,12.6,110.000000
2015,17.1,100.000000
2016,15.7,93.000000
2017,20.8,82.000000
2018,31.2,71.000000
2019,42.3,60.000000
2020,51.6,53.333333


In [311]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(x=df.index.values, y=df['% Newly Registered Cars'], 
           marker_color=['rgba(0,0,100,1)' if x > 50 else 'rgba(10,70,150,.3)' for x in df['% Newly Registered Cars'].values],
                            hovertemplate="%{x}: %{y}%",
          name='EV Sales'),
    secondary_y=False)

fig.update_yaxes(showgrid=False, showticklabels=False, secondary_y=False)
fig.update_layout(template='plotly_white')


fig.add_trace(
    go.Scatter(x=df.index.values, y=df['Avg fleet intensity'], name="Fleet Intensity, New Sales", 
               text=df['Avg fleet intensity'],
               mode='lines+markers+text',
               texttemplate='%{text:1d}',
               marker_color='green', 
               line = dict(width=3),
               textfont=dict(color='green'),
              textposition='top center',
                  hovertemplate="%{x}: %{y} gCO2/km"), 
    secondary_y=True,

)
# fig.update_traces(textposition='top left')

fig.update_layout(
    title_text="More than 50% of new cars sold in Norway are EV's, <br> yielding fleet emissions 10 years ahead of EU targets",
    titlefont=dict(size=25),
    yaxis=dict(title_text="<b>EVs, % of new cars sold</b>", 
               side='right',
               titlefont=dict(color='rgba(0,0,100,1)',size=15)
              ),
    yaxis2=dict(title_text="<b>gCO2/km</b> of fleet",
                tickfont=dict(color='green'),
                                titlefont=dict(color='green',size=15),
               side='left'),
    showlegend=False,

)


fig.update_yaxes(showgrid=False, showticklabels=False, secondary_y=True)

fig.add_trace(
    go.Scatter(x=[2011, 2019], y=[target_2030, target_2030], name="New Fleet Intensity, gCO2/km", 
               opacity=.5, mode='lines',
              line=dict(color='green', dash='dot')), secondary_y=True)
fig.add_annotation(text="EU Target 2030", x=2011, y=9, showarrow=False, secondary_y=True, font=dict(color='green'))


fig.add_annotation(text="<b> 51.6% </b>", x=2020, y=53, showarrow=False, font=dict(size=15))

fig.update_layout(font_family='Graphik, Futura',
                 xaxis=dict(tickvals=[2011, 2015, 2020]))

# fig.write_html("NorwayEVs.html")